In [1]:
import geopandas as gpd
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:

## En esta notebook, se trae la informacion de Pobreza del ultimo anio. 

In [3]:
frac = 0.02
# data = pd.read_csv('./../data/Pobreza/pobreza_'+str(frac)+'_2020.csv')
data = pd.read_csv('./../data/Pobreza/pobreza_'+str(frac)+'_2019-2020.csv')
# data['Grupo Etario'] = pd.cut(data.P03, np.arange(-1, 80, 3))#.round(-1) ## Customizar grupo etario


radio_ref = pd.read_csv('./../data/info/radio_ref.csv')
# radio_ref['DPTO'] = radio_ref['DPTO'].astype(int)
radio_ref['COD_2010'] = radio_ref['radio'].astype(str).str.zfill(9)
# radio_ref[['PROV','NOMPROV','DPTO', 'NOMDPTO']].drop_duplicates().to_csv('./data/DPTO_PROV.csv', index = False)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)
aglo_labels = pd.read_csv('./../data/info/aglo_labels.csv')[['AGLOMERADO', 'NOMAGLO']]
radio_ref = radio_ref[['RADIO_REF_ID', 'COD_2010', 'NOMDPTO', 'AGLOMERADO', 'Region']].drop_duplicates().merge(aglo_labels)


data = data.merge(radio_ref, on = ['RADIO_REF_ID', 'AGLOMERADO'])
data['AGLO_si'] = data.AGLOMERADO != 0

## Reescalar para dar precios actualizados.

In [4]:
cpi = pd.read_csv('./../data/info/indice_precios_d.csv', index_col=0)

from datetime import datetime
hoy = datetime.today().strftime('%Y-%m-%d')
# hoy = '2021-06-01'

# Ratio de precios de hoy con respecto a precios con indice en base modelo
ix = cpi.loc[hoy].values[0]/cpi.loc['2016-01-01'].values[0]
# ix = cpi.loc['2020-09-30'].values[0]/cpi.loc['2016-01-01'].values[0] # Precios del periodo en estudio
ix

columnas_pesos =  ['P47T_persona', 'P47T_hogar', 'CBA', 'gap_indigencia', 'CBT', 'gap_pobreza']
data[columnas_pesos] = ix*data[columnas_pesos]

In [5]:
hoy

'2021-08-02'

### Synthetizer functions

In [6]:
### Sintetizadores
import datetime as dt
import numpy as np

## Base Personas
def sintetizar_P(data, grouper, frac = 0.02):
    df = data.copy()
    df['Total'] = True
    df['timestamp'] = dt.datetime.today()
    n_Qs = df.Q.nunique()
    
#     # Funciones percentil
#     percentiles = [.10, .25, .75, .9, .2]
#     quantile_funcs = [(p, lambda x: x.quantile(p)) for p in percentiles]

    df = df.groupby(grouper + ['timestamp'])[['Total', 'Pobreza', 'Indigencia', 'Pobreza_5usd','P47T_persona']].agg(
        {'Total': ['mean', 'sum'], 'Pobreza': ['mean', 'sum'], 'Indigencia': ['mean', 'sum'], 'Pobreza_5usd': ['mean', 'sum'], 
         'P47T_persona':['mean', q10, q25, 'median', q75, q90]})
    
    df[[('Total', 'sum')]] = (df[[('Total', 'sum')]]/frac).round(1)#.astype(int)
    df[[('Pobreza', 'sum')]] = (df[[('Pobreza', 'sum')]]/frac).round(1)#.astype(int)
    df[[('Indigencia', 'sum')]] = (df[[('Indigencia', 'sum')]]/frac).round(1)#.astype(int)
    df[[('Pobreza_5usd', 'sum')]] = (df[[('Pobreza_5usd', 'sum')]]/frac).round(1)#.astype(int)
    df[[('Total', 'mean')]] = df[[('Total', 'mean')]].round(4)
    df[[('Indigencia', 'mean')]] = df[[('Indigencia', 'mean')]].round(4)
    df[[('Pobreza', 'mean')]] = df[[('Pobreza', 'mean')]].round(4)
    df[[('Pobreza_5usd', 'mean')]] = df[[('Pobreza_5usd', 'mean')]].round(4)
    
    df['P47T_persona'] = df['P47T_persona'].round(-1).astype(int)
    
    dfB = df.T.set_index(np.repeat('Personas', df.shape[1]), append=True)
       
#     dfB = dfB.stack().stack().stack().reset_index()
    stacker_ix = [-i for i in range(len(grouper) + 1)]
    dfB = dfB.stack(level = stacker_ix).reset_index()
    
    dfB = dfB.rename(columns = {'level_0': 'observable', 'level_1': 'sintetico', 'level_2': 'base', 0: 'valor'})
    dfB['valor'] = dfB['valor'].astype(float)
    return dfB

def sintetizar_H(data, grouper, frac = 0.02):
    df = data.copy()
    df = df.groupby('HOGAR_REF_ID').first()
    df['Total'] = True
    df['timestamp'] = dt.datetime.today()
    n_Qs = df.Q.nunique()

#     # Funciones percentil
#     percentiles = [.10, .25, .75, .9, .2]
#     quantile_funcs = [(p, lambda x: x.quantile(p)) for p in percentiles]

    df = df.groupby(grouper + ['timestamp'])[['Total', 'Pobreza', 'Indigencia', 'Pobreza_5usd', 'P47T_hogar', 'CB_EQUIV', 'CBA', 
                                              'gap_indigencia', 'CBT', 'gap_pobreza']
                            ].agg(
    {'Total': ['mean', 'sum'], 'Pobreza': ['mean', 'sum'], 'Indigencia': ['mean', 'sum'], 'Pobreza_5usd': ['mean', 'sum'], 
     'P47T_hogar': ['mean', q10, q25, 'median', q75, q90],
     'CB_EQUIV': ['mean', 'median'],     
     'CBA': ['sum', 'median'], 'gap_indigencia': ['mean', 'median'], 
     'CBT': ['sum', 'median'], 'gap_pobreza': ['mean', 'median']})
        
    df[[('Total', 'sum')]] = (df[[('Total', 'sum')]]/frac/n_Qs).round(1)#.astype(int)
    df[[('Pobreza', 'sum')]] = (df[[('Pobreza', 'sum')]]/frac/n_Qs).round(1)#.astype(int)
    df[[('Indigencia', 'sum')]] = (df[[('Indigencia', 'sum')]]/frac/n_Qs).round(1)#.astype(int)
    df[[('Pobreza_5usd', 'sum')]] = (df[[('Pobreza_5usd', 'sum')]]/frac).round(1)#.astype(int)
    df[[('Total', 'mean')]] = df[[('Total', 'mean')]].round(4)
    df[[('Pobreza', 'mean')]] = df[[('Pobreza', 'mean')]].round(4)
    df[[('Indigencia', 'mean')]] = df[[('Indigencia', 'mean')]].round(4)
    df[[('Pobreza_5usd', 'mean')]] = (df[[('Pobreza_5usd', 'mean')]]/frac).round(1)#.astype(int)

    df['CB_EQUIV'] = df['CB_EQUIV'].round(4)
    
    df['P47T_hogar'] = df['P47T_hogar'].round(-1).astype(int)
    df['CBA'] = df['CBA'].round(-1).astype(int)
    df['gap_indigencia'] = df['gap_indigencia'].round(-1).astype(int)
    df['CBT'] = df['CBT'].round(-1).astype(int)
    df['gap_pobreza'] = df['gap_pobreza'].round(-1).astype(int)
    
    dfC = df.T.set_index(np.repeat('Hogares', df.shape[1]), append=True)
    dfC.index.names = ['observable', 'sintetico', 'base']
    
#     FrozenList(['observable', 'sintetico', 'base', 'timestamp', 'AGLOMERADO', 'Region', 'Q'])
    
# #     dfC = dfC.stack().stack().stack().reset_index()
    stacker_ix = [-i for i in range(len(grouper) + 1)]
    dfC = dfC.stack(level = stacker_ix).reset_index()
#     # es como df.stack([-1, -2, -3, -4]) pero se adapta a groupers mas o menos largos
#     dfC = dfC.stack(stacker_ix).reset_index()

    dfC = dfC.rename(columns = {'level_0': 'observable', 'level_1': 'sintetico', 'level_2': 'base', 0: 'valor'})
    dfC['valor'] = dfC['valor'].astype(float)
    return dfC

# Percentile funcs
def q10(x):
    return x.quantile(0.1)

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

def q90(x):
    return x.quantile(0.9)

In [7]:
sintetizar_P(data, ['PROV'], frac = 0.02).groupby(['observable','sintetico','base']).size()

observable    sintetico  base    
Indigencia    mean       Personas    24
              sum        Personas    24
P47T_persona  mean       Personas    24
              median     Personas    24
              q10        Personas    24
              q25        Personas    24
              q75        Personas    24
              q90        Personas    24
Pobreza       mean       Personas    24
              sum        Personas    24
Pobreza_5usd  mean       Personas    24
              sum        Personas    24
Total         mean       Personas    24
              sum        Personas    24
dtype: int64

In [8]:
# sintetizar_H(data, ['PROV'], frac = 0.02).groupby(['observable','sintetico','base']).size()

In [9]:
# # Ej. Personas en pobreza
# # df = sintetizar_P(data, ['PROV'], frac = 0.02)
# # df.loc[df.sintetico == 'sum'].groupby('observable')['valor'].sum() 

# Indigencia     6500687.5
# Pobreza       21147900.0
# Total         44603650.0

## Los niveles geograficos disponibles son:
 - Radios (RADIO_REF_ID)
 - Fracciones (IDFRAC, no es clave unica)
 - Dptos (DPTO)
 - Provs (PROV)
 - Aglos (AGLOMERADO)
 - (Region)

### Cargar geometrias

In [10]:
## RADIO REF. Referencia de radios censales segun Censo 2010

# radio_ref = pd.read_csv('./radio_ref.csv')
# radio_ref['DPTO'] = radio_ref['DPTO'].astype(int)
# radio_ref['COD_2010'] = radio_ref['radio'].astype(str).str.zfill(9)

## POLIGONOS
# admin410_f = './../../IGN_INDEC_georef/censos_shp_CONICET/2010_RADIOS ARGENTINA'
admin310_f = './../../IGN_INDEC_georef/censos_shp_CONICET_dissolved/fracs_2010.shp'
admin210_f = './../../IGN_INDEC_georef/censos_shp_CONICET_dissolved/dptos_2010.shp'
admin1_f = './../../IGN_INDEC_georef/IGN_shp/Provincia/'

## Provincias del IGN
admin1 = gpd.read_file(admin1_f)  # COD_2010 = '020130302'
# admin1['ID'] = admin1.IN1 + '0000000'
admin1[['PROV']] = admin1.IN1.astype(int)
admin1 = admin1[['PROV', 'geometry']]

### Poligonos de censo de CONICET.
# admin410 = gpd.read_file(admin410_f)  # COD_2010 = '020130302'
# admin410['ID'] = admin410['COD_2010']

admin310 = gpd.read_file(admin310_f)  
admin310['IDFRAC'] = admin310.PROV_ + admin310.DEPTO_ + admin310.FRACC_ + '00'
admin310 = admin310[['IDFRAC', 'geometry']]

admin210 = gpd.read_file(admin210_f) 
# admin210['ID'] = admin210.PROV_ + admin210.DEPTO_ + '0000'
admin210['DPTO'] = (admin210['PROV_'] + admin210['DEPTO_']).astype(int)
admin210 = admin210[['DPTO', 'geometry']]


## ADOPTAR CRS del IGN
admin210 = admin210.to_crs(admin1.crs)
admin310 = admin310.to_crs(admin1.crs)

In [11]:
admin1['area_km2'] = admin1['geometry'].to_crs('epsg:3395').map(lambda p: p.area / 10**6)
admin210['area_km2'] = admin210['geometry'].to_crs('epsg:3395').map(lambda p: p.area / 10**6)
admin310['area_km2'] = admin310['geometry'].to_crs('epsg:3395').map(lambda p: p.area / 10**6)

In [12]:
admin310.head()

,IDFRAC,geometry,area_km2
0,020010100,"POLYGON ((-58.37431 -34.57871, -58.37431 -34.5...",2.663001
1,020010200,"POLYGON ((-58.37567 -34.58419, -58.37552 -34.5...",0.627842
2,020010300,"POLYGON ((-58.38788 -34.58285, -58.38788 -34.5...",1.302769
3,020010400,"POLYGON ((-58.38650 -34.59284, -58.38650 -34.5...",0.299640
4,020010500,"POLYGON ((-58.38161 -34.59284, -58.38161 -34.5...",0.294890


### Funcion guardar geojson

In [13]:
import os
# pip install geopandas
import geopandas as gpd


def save_geojson(gdf, filename = 'test.geojson'):
    if not os.path.exists('./../data/geojson/'):
        os.makedirs('./../data/geojson/')
    
    try:    # Delete if geojson exists, cause overwriting is not supported
        os.remove('./../data/geojson/'+filename)
    except OSError:
        pass
    
    gdf.to_file('./../data/geojson/'+filename, driver="GeoJSON", encoding='utf-8')


In [14]:
# # Las variables agrupadoras pueden ser: 
# groupers = [['Q','Total_pais'], ['Q','AGLOMERADO'], ['Q','Region'], 
#             ['Q','PROV'], ['Q','DPTO']]

## Base: Personas

In [15]:
## PROVINCIAS

df = sintetizar_P(data, ['PROV'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin1), crs = admin1.crs)

save_geojson(gdf, filename = 'pobreza_P_PROV.geojson')

In [16]:
gdf.head(1)

,base,PROV,Total_mean,Total_sum,Pobreza_mean,Pobreza_sum,Indigencia_mean,Indigencia_sum,Pobreza_5usd_mean,Pobreza_5usd_sum,P47T_persona_mean,P47T_persona_q10,P47T_persona_q25,P47T_persona_median,P47T_persona_q75,P47T_persona_q90,geometry,area_km2
0,Personas,2,1.0,23572600.0,0.1851,4362750.0,0.0474,1117150.0,0.0369,869300.0,54010.0,0.0,0.0,41050.0,78760.0,123570.0,"POLYGON Z ((-58.34189 -34.63110 0.00000, -58.3...",303.401389


In [17]:
## DPTOS
df = sintetizar_P(data, ['DPTO'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin210), crs = admin210.crs)

save_geojson(gdf, filename = 'pobreza_P_DPTO.geojson')

In [18]:
gdf.head(1)

,base,DPTO,Total_mean,Total_sum,Pobreza_mean,Pobreza_sum,Indigencia_mean,Indigencia_sum,Pobreza_5usd_mean,Pobreza_5usd_sum,P47T_persona_mean,P47T_persona_q10,P47T_persona_q25,P47T_persona_median,P47T_persona_q75,P47T_persona_q90,geometry,area_km2
0,Personas,2001,1.0,1727000.0,0.2246,387950.0,0.071,122650.0,0.0565,97500.0,54630.0,0.0,0.0,41780.0,78540.0,123570.0,"POLYGON ((-58.36796 -34.62520, -58.36796 -34.6...",26.223521


In [ ]:
## FRACS
data['IDFRAC'] = data['COD_2010'].str[:-2] + '00'
df = sintetizar_P(data, ['IDFRAC'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()

df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin310), crs = admin310.crs)

save_geojson(gdf, filename = 'pobreza_P_FRAC.geojson')

## Base: Personas, Mayores de 24.

In [10]:
## PROVINCIAS

df = sintetizar_P(data.loc[data.P03 >= 24], ['PROV'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin1), crs = admin1.crs)

save_geojson(gdf, filename = 'pobreza_M24_PROV.geojson')

In [11]:
gdf.head(1)

,base,PROV,Total_mean,Total_sum,Pobreza_mean,Pobreza_sum,Indigencia_mean,Indigencia_sum,Pobreza_5usd_mean,Pobreza_5usd_sum,P47T_persona_mean,P47T_persona_q10,P47T_persona_q25,P47T_persona_median,P47T_persona_q75,P47T_persona_q90,geometry,area_km2
0,Personas,2,1.0,16806800.0,0.1251,2102400.0,0.0272,457100.0,0.0209,351450.0,72530.0,13090.0,32230.0,59340.0,95250.0,141770.0,"POLYGON Z ((-58.34189 -34.63110 0.00000, -58.3...",303.401389


In [12]:
## DPTOS
df = sintetizar_P(data.loc[data.P03 >= 24], ['DPTO'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin210), crs = admin210.crs)

save_geojson(gdf, filename = 'pobreza_M24_DPTO.geojson')

In [13]:
gdf.head(1)

,base,DPTO,Total_mean,Total_sum,Pobreza_mean,Pobreza_sum,Indigencia_mean,Indigencia_sum,Pobreza_5usd_mean,Pobreza_5usd_sum,P47T_persona_mean,P47T_persona_q10,P47T_persona_q25,P47T_persona_median,P47T_persona_q75,P47T_persona_q90,geometry,area_km2
0,Personas,2001,1.0,1233400.0,0.1482,182800.0,0.0396,48900.0,0.0319,39350.0,72620.0,11810.0,31420.0,59070.0,94710.0,141770.0,"POLYGON ((-58.36796 -34.62520, -58.36796 -34.6...",26.223521


In [ ]:
## FRACS
data['IDFRAC'] = data['COD_2010'].str[:-2] + '00'
df = sintetizar_P(data.loc[data.P03 >= 24], ['IDFRAC'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()

df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin310), crs = admin310.crs)

save_geojson(gdf, filename = 'pobreza_M24_FRAC.geojson')

## Base: Personas, Menores de 14.

In [12]:
## PROVINCIAS

df = sintetizar_P(data.loc[data.P03 <= 14], ['PROV'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin1), crs = admin1.crs)

save_geojson(gdf, filename = 'pobreza_M14_PROV.geojson')

In [13]:
gdf.head(1)

,base,PROV,Total_mean,Total_sum,Pobreza_mean,Pobreza_sum,Indigencia_mean,Indigencia_sum,Pobreza_5usd_mean,Pobreza_5usd_sum,P47T_persona_mean,P47T_persona_q10,P47T_persona_q25,P47T_persona_median,P47T_persona_q75,P47T_persona_q90,geometry,area_km2
0,Personas,2,1.0,3915800.0,0.365,1429350.0,0.104,407050.0,0.0806,315500.0,80.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-58.34189 -34.63110 0.00000, -58.3...",303.401389


In [14]:
## DPTOS
df = sintetizar_P(data.loc[data.P03 <= 14], ['DPTO'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin210), crs = admin210.crs)

save_geojson(gdf, filename = 'pobreza_M14_DPTO.geojson')

In [15]:
gdf.head(1)

,base,DPTO,Total_mean,Total_sum,Pobreza_mean,Pobreza_sum,Indigencia_mean,Indigencia_sum,Pobreza_5usd_mean,Pobreza_5usd_sum,P47T_persona_mean,P47T_persona_q10,P47T_persona_q25,P47T_persona_median,P47T_persona_q75,P47T_persona_q90,geometry,area_km2
0,Personas,2001,1.0,270800.0,0.4614,124950.0,0.1586,42950.0,0.1263,34200.0,80.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-58.36796 -34.62520, -58.36796 -34.6...",26.223521


In [ ]:
## FRACS
data['IDFRAC'] = data['COD_2010'].str[:-2] + '00'
df = sintetizar_P(data.loc[data.P03 <= 14], ['IDFRAC'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()

df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin310), crs = admin310.crs)

save_geojson(gdf, filename = 'pobreza_M14_FRAC.geojson')

## Base: Personas, Menores de 6.

In [16]:
## PROVINCIAS

df = sintetizar_P(data.loc[data.P03 <= 6], ['PROV'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin1), crs = admin1.crs)

save_geojson(gdf, filename = 'pobreza_M6_PROV.geojson')

In [17]:
gdf.head(1)

,base,PROV,Total_mean,Total_sum,Pobreza_mean,Pobreza_sum,Indigencia_mean,Indigencia_sum,Pobreza_5usd_mean,Pobreza_5usd_sum,P47T_persona_mean,P47T_persona_q10,P47T_persona_q25,P47T_persona_median,P47T_persona_q75,P47T_persona_q90,geometry,area_km2
0,Personas,2,1.0,1902800.0,0.3586,682300.0,0.103,196050.0,0.0806,153400.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-58.34189 -34.63110 0.00000, -58.3...",303.401389


In [18]:
## DPTOS
df = sintetizar_P(data.loc[data.P03 <= 6], ['DPTO'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin210), crs = admin210.crs)

save_geojson(gdf, filename = 'pobreza_M6_DPTO.geojson')

In [19]:
gdf.head(1)

,base,DPTO,Total_mean,Total_sum,Pobreza_mean,Pobreza_sum,Indigencia_mean,Indigencia_sum,Pobreza_5usd_mean,Pobreza_5usd_sum,P47T_persona_mean,P47T_persona_q10,P47T_persona_q25,P47T_persona_median,P47T_persona_q75,P47T_persona_q90,geometry,area_km2
0,Personas,2001,1.0,139600.0,0.4724,65950.0,0.1748,24400.0,0.1422,19850.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-58.36796 -34.62520, -58.36796 -34.6...",26.223521


In [ ]:
## FRACS
data['IDFRAC'] = data['COD_2010'].str[:-2] + '00'
df = sintetizar_P(data.loc[data.P03 <= 6], ['IDFRAC'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()

df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin310), crs = admin310.crs)

save_geojson(gdf, filename = 'pobreza_M6_FRAC.geojson')

## Base: Hogares

In [ ]:
## PROVINCIAS

df = sintetizar_H(data, ['PROV'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin1), crs = admin1.crs)

save_geojson(gdf, filename = 'pobreza_H_PROV.geojson')

In [ ]:
gdf.head(1)

In [ ]:
## DPTOS
df = sintetizar_H(data, ['DPTO'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()
df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin210), crs = admin210.crs)

save_geojson(gdf, filename = 'pobreza_H_DPTO.geojson')

In [ ]:
gdf.head(1)

In [ ]:
## FRACS
data['IDFRAC'] = data['COD_2010'].str[:-2] + '00'
df = sintetizar_H(data, ['IDFRAC'], frac = 0.02).drop('timestamp', axis = 1)#.groupby(['observable','sintetico','base']).size()

df = df.set_index(list(df.drop('valor', 1).columns)).unstack([0, 1])['valor']
df.columns = ['_'.join(col) for col in df.columns.values]
df = df.reset_index()

gdf = gpd.GeoDataFrame(df.merge(admin310), crs = admin310.crs)

save_geojson(gdf, filename = 'pobreza_H_FRAC.geojson')

In [ ]:
xx

In [ ]:
## Repetir para Hogares, para M24, M14, etc.

In [ ]:
cd /media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data
export MAPBOX_ACCESS_TOKEN=sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA

In [ ]:

## HOGARES. 
# (REPEAT FOR PERSONAS, M24, ETC.)
tilesets upload-source matuteiglesias2 income_provs_H ./geojson/pobreza_H_PROV.geojson
tilesets upload-source matuteiglesias2 income_dptos_H ./geojson/pobreza_H_DPTO.geojson
tilesets upload-source matuteiglesias2 income_fracs_H ./geojson/pobreza_H_FRAC.geojson

# # tilesets create username.hello-world-tiles --recipe hello-world-recipe.json --name "hello world"
tilesets create matuteiglesias2.income_H --recipe ./geojson/pobreza_H-recipe.json --name "Ingresos y Pobreza - Hogares"
tilesets publish matuteiglesias2.income_H


In [ ]:
## REPEAT FOR PERSONAS
tilesets upload-source matuteiglesias2 income_provs_P ./geojson/pobreza_P_PROV.geojson
tilesets upload-source matuteiglesias2 income_dptos_P ./geojson/pobreza_P_DPTO.geojson
tilesets upload-source matuteiglesias2 income_fracs_P ./geojson/pobreza_P_FRAC.geojson

tilesets create matuteiglesias2.income_P --recipe ./geojson/pobreza_P-recipe.json --name "Ingresos y Pobreza - Personas"
tilesets publish matuteiglesias2.income_P


In [ ]:
## REPEAT FOR Mayores de 24
tilesets delete matuteiglesias2.income_M24

tilesets upload-source matuteiglesias2 income_provs_M24 ./geojson/pobreza_M24_PROV.geojson
tilesets upload-source matuteiglesias2 income_dptos_M24 ./geojson/pobreza_M24_DPTO.geojson
tilesets upload-source matuteiglesias2 income_fracs_M24 ./geojson/pobreza_M24_FRAC.geojson

tilesets create matuteiglesias2.income_M24 --recipe ./geojson/pobreza_M24-recipe.json --name "Ingresos y Pobreza - Mayores de 24"
tilesets publish matuteiglesias2.income_M24


In [ ]:
## REPEAT FOR Menores de 14
tilesets upload-source matuteiglesias2 income_provs_M14 ./geojson/pobreza_M14_PROV.geojson
tilesets upload-source matuteiglesias2 income_dptos_M14 ./geojson/pobreza_M14_DPTO.geojson
tilesets upload-source matuteiglesias2 income_fracs_M14 ./geojson/pobreza_M14_FRAC.geojson

tilesets create matuteiglesias2.income_M14 --recipe ./geojson/pobreza_M14-recipe.json --name "Ingresos y Pobreza - Menores de 14"
tilesets publish matuteiglesias2.income_M14


In [ ]:
## REPEAT FOR Menores de 14
tilesets upload-source matuteiglesias2 income_provs_M6 ./geojson/pobreza_M6_PROV.geojson
tilesets upload-source matuteiglesias2 income_dptos_M6 ./geojson/pobreza_M6_DPTO.geojson
tilesets upload-source matuteiglesias2 income_fracs_M6 ./geojson/pobreza_M6_FRAC.geojson

tilesets create matuteiglesias2.income_M6 --recipe ./geojson/pobreza_M6-recipe.json --name "Ingresos y Pobreza - Menores de 6"
tilesets publish matuteiglesias2.income_M6


In [ ]:

# # tilesets upload-source username populated-places-source /path/to/data/populated_places.geojson.ld

# tilesets upload-source matuteiglesias2 income_provs ./geojson/pobreza_P_PROV.geojson
# tilesets upload-source matuteiglesias2 income_dptos ./geojson/pobreza_P_DPTO.geojson
# tilesets upload-source matuteiglesias2 income_fracs ./geojson/pobreza_P_FRAC.geojson


# tilesets delete-source matuteiglesias2 s


In [ ]:
# gdf = gpd.read_file('./../data/geojson/pobreza_H_PROV.geojson')
# gdf

In [ ]:
# export MAPBOX_ACCESS_TOKEN=sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA

# # tilesets upload-source username populated-places-source /path/to/data/populated_places.geojson.ld
# cd /media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data

# tilesets upload-source matuteiglesias2 pobreza_v2 ./geojson/pobreza_P_PROV.geojson
# tilesets upload-source matuteiglesias2 pobreza_v2 ./geojson/pobreza_P_DPTO.geojson

# # tilesets create username.hello-world-tiles --recipe hello-world-recipe.json --name "hello world"
# tilesets create matuteiglesias2.personas2 --recipe ./geojson/pobreza-recipe.json --name "Ingresos y Pobreza - Personas2"
# tilesets publish matuteiglesias2.personas2


In [1]:
>>> import seaborn as sns
>>> pal = sns.color_palette("RdYlGn_r", 10)
>>> print(pal.as_hex())


['#17934e', '#57b65f', '#93d168', '#c7e77f', '#eef8a8', '#fff1a8', '#fece7c', '#fa9b58', '#ee613e', '#d22b27']


In [2]:
>>> import seaborn as sns
>>> pal = sns.color_palette("cool", 10)
>>> print(pal.as_hex())

['#17e8ff', '#2ed1ff', '#45baff', '#5da2ff', '#748bff', '#8b74ff', '#a25dff', '#ba45ff', '#d12eff', '#e817ff']
